
**3-Engenharia-de-Dados-MVP-Criação-Carga-Gold**

Este notebook cria e carrega as tabelas da camada golden a partir das tabelas da camada silver. A tabela "cursos_final" é criada a partir da tabela "cursos" (camada silver), acrescida de campos que descrevem unidade de ensino e são provenientes da tabela "escolas" (camada silver). Não serão carregados cursos, cuja unidade de ensino não exista na tabela "escolas" (camada silver).

A tabela "escolas_final", por sua vez, é criada a partir da tabela "escolas" (camada silver) e de uma tabela temporária da camada golden que possui quantitativos de cursos por unidade de ensino. Essa tabela, "escolas_qtd_cursos", é apagada ao final.

In [0]:
%sql

-- 2) Criando camada gold

-- criando camada gold
CREATE DATABASE gold


In [0]:
%sql

-- 3) Criando tabela cursos_final com campos selecionados e detalhes da unidade de ensino

-- Como as informações de UNIDADE_DE_ENSINO, UF, CODIGO_MUNICIPIO e MUNICIPIO são provenientes da tabela escolas (silver), será mantido apenas um campo para cada na tabela cursos_final (gold), resultado da junção entre cursos (silver) e escolas (silver); os campos de mesmo nome na tabela cursos (silver) serão desconsiderados.

CREATE TABLE gold.cursos_final (
    NOME_SUBTIPO_DE_CURSOS VARCHAR(255),
    CODIGO_CURSO INT,
    CURSO VARCHAR(255),
    EIXO_TECNOLOGICO VARCHAR(255),
    MODALIDADE VARCHAR(100),
    CARGA_HORARIA_CURSO INT,
    SITUACAO_ATIVO BOOLEAN,
    NOME_SISTEMA_DE_ENSINO VARCHAR(255),
    NOME_TIPO_ESCOLA VARCHAR(255),
    CODIGO_UNIDADE_DE_ENSINO INT,
    UNIDADE_DE_ENSINO VARCHAR(255),
    NOME_DEPENDENCIA_ADMINISTRATIVA VARCHAR(255),
    NOME_SUBDEPENDENCIA_ADMINISTRATIVA VARCHAR(255),
    UF CHAR(2),
    CODIGO_MUNICIPIO INT,
    MUNICIPIO VARCHAR(255)
);


In [0]:
%sql 

-- 4) Povoando tabela cursos_final com dados da junção de cursos e escolas

INSERT INTO gold.cursos_final (
  NOME_SUBTIPO_DE_CURSOS,
  CODIGO_CURSO,
  CURSO,
  EIXO_TECNOLOGICO,
  MODALIDADE,
  CARGA_HORARIA_CURSO,
  SITUACAO_ATIVO,
  NOME_SISTEMA_DE_ENSINO,
  NOME_TIPO_ESCOLA,
  CODIGO_UNIDADE_DE_ENSINO,
  UNIDADE_DE_ENSINO,
  NOME_DEPENDENCIA_ADMINISTRATIVA,
  NOME_SUBDEPENDENCIA_ADMINISTRATIVA,
  UF,
  CODIGO_MUNICIPIO,
  MUNICIPIO)
SELECT 
  sc.NOME_SUBTIPO_DE_CURSOS,
  sc.CODIGO_CURSO,
  sc.CURSO,
  sc.EIXO_TECNOLOGICO,
  sc.MODALIDADE,
  sc.CARGA_HORARIA_CURSO,
  sc.SITUACAO_ATIVO,
  se.NOME_SISTEMA_DE_ENSINO,
  se.NOME_TIPO_ESCOLA,
  se.CODIGO_UNIDADE_DE_ENSINO,
  se.UNIDADE_DE_ENSINO,
  se.NOME_DEPENDENCIA_ADMINISTRATIVA,
  se.NOME_SUBDEPENDENCIA_ADMINISTRATIVA,
  se.UF,
  se.CODIGO_MUNICIPIO,
  se.MUNICIPIO
FROM silver.cursos sc
INNER JOIN silver.escolas se ON sc.CODIGO_UNIDADE_DE_ENSINO = se.CODIGO_UNIDADE_DE_ENSINO


num_affected_rows,num_inserted_rows
83137,83137


In [0]:
%sql

-- 5) Criando tabela temporária escolas_qtd_cursos com quantitativos de cursos

CREATE TABLE gold.escolas_qtd_cursos (
    CODIGO_UNIDADE_DE_ENSINO INT,
    QTD_CURSOS_PRESENCIAIS INT,
    QTD_CURSOS_DISTANCIA INT,
    QTD_CURSOS_TOTAIS INT
);

INSERT INTO gold.escolas_qtd_cursos (
        CODIGO_UNIDADE_DE_ENSINO, 
        QTD_CURSOS_PRESENCIAIS, 
        QTD_CURSOS_DISTANCIA, 
        QTD_CURSOS_TOTAIS)
SELECT se.CODIGO_UNIDADE_DE_ENSINO, 
        COUNT(CASE WHEN sc.MODALIDADE IN ('Educação Presencial', 'Todos') THEN 1 END) AS QTD_CURSOS_PRESENCIAIS,
        COUNT(CASE WHEN sc.MODALIDADE IN ('Educação a Distância', 'Todos') THEN 1 END) AS QTD_CURSOS_DISTANCIA,
        COUNT(sc.CODIGO_CURSO) AS QTD_CURSOS_TOTAIS
FROM silver.escolas se
LEFT JOIN silver.cursos sc ON sc.CODIGO_UNIDADE_DE_ENSINO = se.CODIGO_UNIDADE_DE_ENSINO
GROUP BY se.CODIGO_UNIDADE_DE_ENSINO
ORDER BY QTD_CURSOS_TOTAIS DESC


num_affected_rows,num_inserted_rows
22508,22508


In [0]:
%sql

-- 6) Criando tabela escolas_final com campos selecionados e quantitativos de cursos

CREATE TABLE gold.escolas_final (
    NOME_SISTEMA_DE_ENSINO VARCHAR(255),
    NOME_TIPO_ESCOLA VARCHAR(255),
    CODIGO_UNIDADE_DE_ENSINO INT,
    UNIDADE_DE_ENSINO VARCHAR(255),
    NOME_DEPENDENCIA_ADMINISTRATIVA VARCHAR(255),
    NOME_SUBDEPENDENCIA_ADMINISTRATIVA VARCHAR(255),
    UF CHAR(2),
    CODIGO_MUNICIPIO INT,
    MUNICIPIO VARCHAR(255),
    QTD_CURSOS_PRESENCIAIS INT, 
    QTD_CURSOS_DISTANCIA INT, 
    QTD_CURSOS_TOTAIS INT
)


In [0]:
%sql

-- 7) Povoando tabela escolas_final com dados da junção de escolas e escolas_qtd_cursos (temporária)

INSERT INTO gold.escolas_final (
    NOME_SISTEMA_DE_ENSINO,
    NOME_TIPO_ESCOLA,
    CODIGO_UNIDADE_DE_ENSINO,
    UNIDADE_DE_ENSINO,
    NOME_DEPENDENCIA_ADMINISTRATIVA,
    NOME_SUBDEPENDENCIA_ADMINISTRATIVA,
    UF,
    CODIGO_MUNICIPIO,
    MUNICIPIO,
    QTD_CURSOS_PRESENCIAIS, 
    QTD_CURSOS_DISTANCIA, 
    QTD_CURSOS_TOTAIS
)
SELECT
    se.NOME_SISTEMA_DE_ENSINO,
    se.NOME_TIPO_ESCOLA,
    se.CODIGO_UNIDADE_DE_ENSINO,
    se.UNIDADE_DE_ENSINO,
    se.NOME_DEPENDENCIA_ADMINISTRATIVA,
    se.NOME_SUBDEPENDENCIA_ADMINISTRATIVA,
    se.UF,
    se.CODIGO_MUNICIPIO,
    se.MUNICIPIO,
    gec.QTD_CURSOS_PRESENCIAIS, 
    gec.QTD_CURSOS_DISTANCIA, 
    gec.QTD_CURSOS_TOTAIS
FROM silver.escolas se LEFT JOIN gold.escolas_qtd_cursos gec ON gec.CODIGO_UNIDADE_DE_ENSINO = se.CODIGO_UNIDADE_DE_ENSINO


num_affected_rows,num_inserted_rows
22508,22508


In [0]:
%sql

-- 8) Apagando tabela temporária escolas_qtd_cursos

DROP TABLE gold.escolas_qtd_cursos
